# Lab 1 - Linear regression from scratch 

### Numan SAHNOU & Matthieu ECCHER

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_excel("Folds5x2_pp.xlsx", sheet_name = None)
df = pd.concat(df, ignore_index=True)

df

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90
...,...,...,...,...,...
47835,15.12,48.92,1011.80,72.93,462.59
47836,33.41,77.95,1010.30,59.72,432.90
47837,15.99,43.34,1014.20,78.66,465.96
47838,17.65,59.87,1018.58,94.65,450.93


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47840 entries, 0 to 47839
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      47840 non-null  float64
 1   V       47840 non-null  float64
 2   AP      47840 non-null  float64
 3   RH      47840 non-null  float64
 4   PE      47840 non-null  float64
dtypes: float64(5)
memory usage: 1.8 MB


In [ ]:
df.describe()

,AT,V,AP,RH,PE
count,47840.000000,47840.000000,47840.000000,47840.000000,47840.000000
mean,19.651231,54.305804,1013.259078,73.308978,454.365009
std,7.452162,12.707362,5.938535,14.599658,17.066281
min,1.810000,25.360000,992.890000,25.560000,420.260000
25%,13.510000,41.740000,1009.100000,63.327500,439.750000
50%,20.345000,52.080000,1012.940000,74.975000,451.550000
75%,25.720000,66.540000,1017.260000,84.830000,468.430000
max,37.110000,81.560000,1033.300000,100.160000,495.760000


# Normalization

### Defintion of features and labels and train/test split of dataset

In [ ]:
df_train, df_test = train_test_split(df, test_size = 0.2, random_state=0)

X_train = df_train[['AT', 'V', 'AP', 'RH']]
y_train = df_train[['PE']]

X_test = df_test[['AT', 'V', 'AP', 'RH']]
y_test = df_test[['PE']]

X = df[['AT', 'V', 'AP', 'RH']]
y = df[['PE']]

### Normalization of the features 

In [ ]:
def normalizeFeatures(df):
    df_norm = df.copy()
    for column in df_norm.columns:
        df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        
    return df_norm

X = normalizeFeatures(X)
X.insert (0, "intercept", np.nan)

X= X.fillna(1)

X_train = normalizeFeatures(X_train)
X_train.insert (0, "intercept", np.nan)

X_train= X_train.fillna(1)
X_train

X_test = normalizeFeatures(X_test)
X_test.insert (0, "intercept", np.nan)

X_test= X_test.fillna(1)

X_train

,intercept,AT,V,AP,RH
14837,1.0,0.298300,0.262278,0.566196,0.894102
2427,1.0,0.607365,0.471530,0.511012,0.707507
21082,1.0,0.480170,0.490036,0.578570,0.524665
41692,1.0,0.711331,0.953025,0.464242,0.947185
17741,1.0,0.762606,0.863879,0.297451,0.597989
...,...,...,...,...,...
21243,1.0,0.619263,0.834164,0.453601,0.869839
45891,1.0,0.366572,0.281139,0.413017,0.823056
42613,1.0,0.653258,0.769573,0.452858,0.403887
43567,1.0,0.210482,0.285409,0.629795,0.795576


In [ ]:
X_train_numpy = X_train.to_numpy()
y_train_numpy = y_train.to_numpy()
m = np.size(X_train,0)
n = np.size(X_train,1)

# Creation of weight 

In [ ]:
import random
def weight(df):
    w = []
    for i in range (0, len(df.axes[1])):
        w.append(random.uniform(-1000, 1000)) 
    np.array(w)
    
    return np.transpose([w])

w = weight(X_train)


# Cost Function

In [ ]:
def costFunction(X_train, y_train, w):
    J = 0
    h = X_train.dot(w)
    J = 1/(2*m) * np.sum(np.square(h-y_train))
    return J

# Gradient descent

In [ ]:
def gradient_descent(X_train, y_train, w, learning_rate, iterations, d):

    cost_history = []
    w_history = np.zeros((iterations,n))
    
    for i in range(iterations):
        h = np.dot(X_train,w)
        w = w - learning_rate * X_train.T.dot((h-y_train))
        w_history[i,:] = w.T
        cost_history.append(costFunction(X_train, y_train, w))

        print('iteration : ', i, ' | cout : ', cost_history[i], ' | difference pred-actual : ', cost_history[i-1]-cost_history[i], ' | delta condition : ', np.absolute(cost_history[i-1]-cost_history[i]) < d, ' | alpha : ', learning_rate)
        
        if (cost_history[i-1] - cost_history[i] < 0):
            learning_rate = learning_rate / 2
        elif(i > 0 and np.absolute(cost_history[i-1] - cost_history[i]) < d):
            return w, cost_history, w_history

    return w, cost_history, w_history

## Application

In [ ]:
alpha = 0.001
iterations = 10000
delta = 0.00001
cost_history = []
w, cost_history, w_history = gradient_descent(X_train_numpy, y_train_numpy, w, alpha, iterations, delta)

print('w0:', w[0][0], '\nw1:', w[1][0], '\nw2:', w[2][0], '\nw3:', w[3][0], '\nw4:', w[4][0])
print('Final cost :', cost_history[-1])


iteration :  1430  | cout :  10.625120929980584  | difference pred-actual :  0.0007864264726293868  | delta condition :  False  | alpha :  1.5625e-05
iteration :  1431  | cout :  10.624337956692399  | difference pred-actual :  0.000782973288185218  | delta condition :  False  | alpha :  1.5625e-05
iteration :  1432  | cout :  10.623558421425514  | difference pred-actual :  0.0007795352668846789  | delta condition :  False  | alpha :  1.5625e-05
iteration :  1433  | cout :  10.622782309083393  | difference pred-actual :  0.0007761123421214933  | delta condition :  False  | alpha :  1.5625e-05
iteration :  1434  | cout :  10.622009604635775  | difference pred-actual :  0.0007727044476180112  | delta condition :  False  | alpha :  1.5625e-05
iteration :  1435  | cout :  10.621240293118401  | difference pred-actual :  0.0007693115173736942  | delta condition :  False  | alpha :  1.5625e-05
iteration :  1436  | cout :  10.62047435963274  | difference pred-actual :  0.0007659334856615629  | 

# Mean Absolute Error

## Formula

In [ ]:
y_predicted = w[0][0] + w[1][0]*(X_test['AT']) + w[2][0]*(X_test['V']) + w[3][0]*(X_test['AP']) + w[4][0]*(X_test['RH'])

y_predicted = y_predicted.to_frame()
y_predicted = y_predicted.to_numpy()
y_actual = y_test.to_numpy()

In [ ]:
m = np.size(X_test,0)
n = np.size(X_test,1)

MAE = np.sum(np.absolute(y_actual - y_predicted)) / m

MAE

3.6171101359552122

# Root Mean Squared Error

In [ ]:
RMSE = np.sqrt( np.sum( np.square(y_predicted - y_actual) / m ))
RMSE


4.544162074942925

# R squared

In [ ]:
y_moy = np.sum(y_actual) / m 
R_squared = 1 - (np.sum(np.square(y_actual - y_predicted)) / np.sum(np.square(y_actual - y_moy)))
R_squared

0.9281213237353031

# Scikit Learn comparison

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)

print(model.coef_, model.intercept_)
coef = model.coef_
intercept =model.intercept_

sci_R_squared = model.score(X_test, y_test)
sci_MAE = mean_absolute_error(y_test, model.predict(X_test))
sci_RMSE =  np.sqrt(mean_squared_error(y_test, model.predict(X_test)))

print('R^2 : ', sci_R_squared)
print('MAE : ', sci_MAE)
print('RMSE : ', sci_RMSE)


[[  0.         -69.76022852 -13.22448548   2.55451842 -11.72429747]] [502.65651823]
R^2 :  0.9280933904852822
MAE :  3.6155251291149244
RMSE :  4.545044957685595


# Comparison 

In [ ]:
d = {'From scikit learn': [model.intercept_[0], model.coef_[0][1], model.coef_[0][2], model.coef_[0][3], model.coef_[0][4], sci_R_squared, sci_MAE, sci_RMSE], 'From scratch': [w[0][0],w[1][0],w[2][0],w[3][0],w[4][0], R_squared, MAE, RMSE]}
comparison = pd.DataFrame(data=d)
comparison['Difference'] = comparison['From scikit learn'] - comparison['From scratch']
comparison.insert (0, "Raw", ['intercept', 'w0', 'w1', 'w2', 'w3', 'R squared', 'MAE', 'RMSE'])
comparison.set_index('Raw')

,From scikit learn,From scratch,Difference
Raw,,,
intercept,502.656518,502.120595,0.535923
w0,-69.760229,-69.016308,-0.743920
w1,-13.224485,-13.645163,0.420678
w2,2.554518,2.889837,-0.335319
w3,-11.724297,-11.403077,-0.321221
R squared,0.928093,0.928121,-0.000028
MAE,3.615525,3.617110,-0.001585
RMSE,4.545045,4.544162,0.000883


### We can observe that we have really close values from scratch than from scikit learn library. 
### We can conclude that we built a good model from scratch.
### 

# Normal equation

In [ ]:
w_normal = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
w_normal

array([[502.6995263 ],
       [-69.80621266],
       [-13.14610295],
       [  2.50877176],
       [-11.79083608]])

### R squared normal equation

In [ ]:
y_normal_predicted = w_normal.T.dot(X_test.to_numpy().T).T

y_moy = np.sum(y_actual) / m 
normal_R_squared = 1 - (np.sum(np.square(y_actual - y_normal_predicted)) / np.sum(np.square(y_actual - y_moy)))
normal_R_squared

0.9281500610444801